## Outline: 

### 1. Giới thiệu và một số lưu ý. 
### 2. Ví dụ trên Python (Iris flower dataset)
### 3. Phương pháp đánh giá (Evaluation method)
### 4. Đánh trọng số cho các điểm lân cận
### 5. Chuẩn hóa DL 
### 6. Ưu điểm của KNN 
### 7. Nhược điểm của KNN

Bắt đầu:

### 1. Giới thiệu và một số lưu ý 

Thuật toán này được xếp vào loại lazy learning.

Với KNN, trong bài toán Classification, label của một điểm dữ liệu mới (hay kết quả của câu hỏi trong bài thi) được suy ra trực tiếp từ K điểm dữ liệu gần nhất trong training set.

Label của một test data có thể được quyết định bằng major voting (bầu chọn theo số phiếu - tức label nào có nhiều điểm nhất thì sẽ được chọn) giữa các điểm gần nhất, hoặc nó có thể được suy ra bằng cách đánh trọng số khác nhau cho mỗi trong các điểm gần nhất đó rồi suy ra label.

Trong bài toán Regresssion, đầu ra của một điểm dữ liệu sẽ bằng chính đầu ra của điểm dữ liệu đã biết gần nhất (trong trường hợp K=1), hoặc là trung bình có trọng số của đầu ra của những điểm gần nhất, hoặc bằng một mối quan hệ dựa trên khoảng cách tới các điểm gần nhất đó.

Một cách ngắn gọn, KNN là thuật toán đi tìm đầu ra của một điểm dữ liệu mới bằng cách chỉ dựa trên thông tin của K điểm dữ liệu trong training set gần nó nhất (K-lân cận), **không quan tâm đến việc có một vài điểm dữ liệu trong những điểm gần nhất này là nhiễu**. 

Có một điều đáng lưu ý là KNN phải nhớ tất cả các điểm dữ liệu training, việc này không được lợi về cả bộ nhớ và thời gian tính toán - giống như khi cậu bạn của chúng ta không tìm được câu trả lời cho câu hỏi cuối cùng.

### 2. Ví dụ trên Python

Ta sẻ sử dụng bộ dữ liệu Iris (Iris flower dataset). Trong phần này, chúng ta sẽ tách 150 dữ liệu trong Iris flower dataset ra thành 2 phần, gọi là training set và test set.

Thuật toán KNN sẽ dựa vào trông tin ở training set để dự đoán xem mỗi dữ liệu trong test set tương ứng với loại hoa nào.

Dữ liệu được dự đoán này sẽ được đối chiếu với loại hoa thật của mỗi dữ liệu trong test set để đánh giá hiệu quả của KNN.

In [4]:
# iris flower dataset có sẵn trong thư viên scikit-learn
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import neighbors, datasets

In [5]:
# load DL và hiển thị vài dữ liệu mẫu: các class được gán nhãn là 0, 1 và 2. 
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

In [6]:
type(iris)     # la mot dictionary rat lon 

sklearn.utils.Bunch

In [10]:
# dem so luong class va so luong data point: 
print('Numer of classes: %d' % len(np.unique(iris_y)))
print('Number of data points: %d' % len(iris_y))

Numer of classes: 3
Number of data points: 150


In [16]:
# lay thu cac sample tu cac class: 
X0 = iris_X[iris_y == 0,:]
print('Samples from class 0:', X0[:5,:])

X1 = iris_X[iris_y == 1,:]
print('Samples from class 1:', X1[:5,:])

X2 = iris_X[iris_y == 2,:]
print('Samples from class 2:', X2[:5,:])

Samples from class 0: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Samples from class 1: [[7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.9 1.5]
 [5.5 2.3 4.  1.3]
 [6.5 2.8 4.6 1.5]]
Samples from class 2: [[6.3 3.3 6.  2.5]
 [5.8 2.7 5.1 1.9]
 [7.1 3.  5.9 2.1]
 [6.3 2.9 5.6 1.8]
 [6.5 3.  5.8 2.2]]


Nhận xét: Nếu nhìn vào vài dữ liệu mẫu, chúng ta thấy rằng hai cột cuối mang khá nhiều thông tin giúp chúng ta có thể phân biệt được chúng (vì dữ liệu có sự khác biệt rất lớn giữa các class). Chúng ta dự đoán rằng kết quả classification cho cơ sở dữ liệu này sẽ tương đối cao.

### Tách training và test set:

In [19]:
# ta dùng 50 điểm DL cho test set, 100 điểm còn lại cho training set. 
# scikit-learn có một hàm số cho phép ta ngẫu nhiên lựa chọn các điểm này: 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size = 50)

print('Training size: %d' % (len(y_train)))
print('Test size: %d' % (len(y_test)))

Training size: 100
Test size: 50


### Ta xét trường hợp đơn giản K = 1 (tức với mỗi điểm test data, ta chỉ xét một điểm training data gần nhất và lấy label của điểm đó để dự đoán cho điểm test này. 

In [20]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 1, p = 2)  # p = 2: khoảng cách theo norm 2.
clf.fit(X_train, y_train)       # train data

# test data: 
y_pred = clf.predict(X_test)

print("Print results for 20 test data points:")
print("Predicted labels: ", y_pred[20:40])
print("Ground truth    : ", y_test[20:40])

Print results for 20 test data points:
Predicted labels:  [1 0 0 2 0 1 0 2 0 2 2 1 1 0 2 2 1 1 2 0]
Ground truth    :  [1 0 0 2 0 1 0 2 0 2 2 1 1 0 2 2 1 1 2 0]


Nhận xét về dự đoán: Kết quả cho thấy label dự đoán gần giống với label thật của test data, chỉ có 2 điểm trong số 20 điểm được hiển thị có kết quả sai lệch.

ground truth: là nhãn/label/đầu ra thực sự của các điểm trong test data.

### 3. Phương pháp đánh giá (evaluation method)

Cách đánh giá: xem xem có bao nhiêu điểm trong test data được dự đoán đúng. Lấy số lượng này chia cho tổng số lượng trong tập test data sẽ ra độ chính xác.

Scikit-learn cung cấp hàm số accuracy_score để thực hiện công việc này.

In [21]:
from sklearn.metrics import accuracy_score

print('Accuracy of 1NN: %.2f %%' %(100*accuracy_score(y_test, y_pred)))  
# bản chất của hàm accuracy_score là phép chia lấy tỉ lệ thông thường

Accuracy of 1NN: 96.00 %


Chú ý: đây là một cơ sở dữ liệu dễ vì chỉ với dữ liệu ở hai cột cuối cùng, chúng ta đã có thể suy ra quy luật.

In [22]:
# thử tính theo norm 1:
from sklearn import neighbors 
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier(n_neighbors = 1, p = 1) 
clf.fit(X_train, y_train)       # train data

# test data: 
y_pred = clf.predict(X_test)

print('Accuracy of 1NN: %.2f %%' %(100*accuracy_score(y_test, y_pred)))  
# bản chất của hàm accuracy_score là phép chia lấy tỉ lệ thông thường

Accuracy of 1NN: 96.00 %


Nhận xét: Norm 1 còn cho độ chính xác cao hơn cả norm 2.

Nhận xét: chỉ xét 1 điểm gần nhất có thể dẫn đến kết quả sai nếu điểm đó là nhiễu. Một cách có thể làm tăng độ chính xác là tăng số lượng điểm lân cận lên, ví dụ 10 điểm, và xem xem trong 10 điểm gần nhất, class nào chiếm đa số thì dự đoán kết quả là class đó. Kỹ thuật dựa vào đa số này được gọi là **major voting**.

In [27]:
from sklearn import neighbors 
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy of 10NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN with major voting: 92.00 %


Nhận xét: 10 điểm dữ liệu có độ chính xác thấp, nghe có vẻ khó tin khi mà dữ liệu ở hai cột cuối cùng phân cụm khá rõ ràng và tách biệt với nhau. Có vẻ như số neighbors lớn quá cũng có thể làm giảm độ chính xác của thuật toán.

In [25]:
# thử với số neighbors = 5

from sklearn import neighbors 
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier(n_neighbors = 5, p = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy of 10NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN with major voting: 96.00 %


Độ chính xác đã được cải thiện.

In [26]:
# n = 3

from sklearn import neighbors 
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy of 10NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN with major voting: 96.00 %


Độ chính xác bằng với n = 5. 

### 4. Đánh trọng số cho các điểm lân cận. 

Trong kỹ thuật major voting bên trên, mỗi trong 10 điểm gần nhất được coi là có vai trò như nhau và giá trị lá phiếu của mỗi điểm này là như nhau. Tôi cho rằng như thế là không công bằng, vì rõ ràng rằng những điểm gần hơn nên có trọng số cao hơn (càng thân cận thì càng tin tưởng). Vậy nên tôi sẽ đánh trọng số khác nhau cho mỗi trong 10 điểm gần nhất này.

Cách đánh trọng số phải thoải mãn điều kiện là một điểm càng gần điểm test data thì phải được đánh trọng số càng cao (tin tưởng hơn). Cách đơn giản nhất là lấy nghịch đảo của khoảng cách này. (Trong trường hợp test data trùng với 1 điểm dữ liệu trong training data, tức khoảng cách bằng 0, ta lấy luôn label của điểm training data).

Scikit-learn giúp chúng ta đơn giản hóa việc này bằng cách gán gía trị weights = 'distance'. (Giá trị mặc định của weights là 'uniform', tương ứng với việc coi tất cả các điểm lân cận có giá trị như nhau như ở trên).

In [29]:
from sklearn import neighbors 
from sklearn.metrics import accuracy_score

# clf là viết tắt của classifier 
clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2, weights = 'distance')   # trọng số = kh/cách

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy of 10NN (1/distance weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN (1/distance weights): 92.00 %


??? ủa, sao lại khác với code mẫu??? code mấu cho kết quả là 100%??? có vẻ như bộ DL sử dụng là khác nhau rồi.

In [36]:
# thử thay đổi số neighbors đi, giảm đi 
from sklearn import neighbors 
from sklearn.metrics import accuracy_score

# clf là viết tắt của classifier 
clf = neighbors.KNeighborsClassifier(n_neighbors = 5, p = 2, weights = 'distance')   # trọng số = kh/cách

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy of 10NN (1/distance weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN (1/distance weights): 98.00 %


### Cách đánh trọng số sử dụng hàm exp

In [41]:
def myweight(distances): 
    sigma2 = .5          # we can change this number 
    return np.exp(-distances ** 2 / sigma2)

clf = neighbors.KNeighborsClassifier(n_neighbors = 5, p = 2, weights = myweight)  # gán hàm vừa định nghĩa ở trên trực tiếp làm đối số luôn 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy of 10NN (customized weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Accuracy of 10NN (customized weights): 96.00 %


Nhận xét: Để đánh giá chính xác hơn kết quả của KNN với K khác nhau, cách định nghĩa khoảng cách khác nhau và cách đánh trọng số khác nhau, chúng ta cần thực hiện quá trình trên với nhiều cách chia dữ liệu training và test khác nhau rồi lấy kết quả trung bình, vì rất có thể dữ liệu phân chia trong 1 trường hợp cụ thể là rất tốt hoặc rất xấu (bias). Đây cũng là cách thường được dùng khi đánh giá hiệu năng của một thuật toán cụ thể nào đó.

### 5. Chuẩn hóa dữ liệu

Data Normalization (chuẩn hóa dữ liệu): đưa các thuộc tính có đơn vị đo khác nhau về cùng một khoảng giá trị, thường là từ 0 đến 1, trước khi thực hiện KNN.

### 6. Ưu điểm của KNN
Độ phức tạp tính toán của quá trình training là bằng 0.
Việc dự đoán kết quả của dữ liệu mới rất đơn giản.
Không cần giả sử gì về phân phối của các class.

### 7. Nhược điểm của KNN
KNN rất nhạy cảm với nhiễu khi K nhỏ.
Như đã nói, KNN là một thuật toán mà mọi tính toán đều nằm ở khâu test. Trong đó việc tính khoảng cách tới từng điểm dữ liệu trong training set sẽ tốn rất nhiều thời gian, đặc biệt là với các cơ sở dữ liệu có số chiều lớn và có nhiều điểm dữ liệu. Với K càng lớn thì độ phức tạp cũng sẽ tăng lên. Ngoài ra, việc lưu toàn bộ dữ liệu trong bộ nhớ cũng ảnh hưởng tới hiệu năng của KNN.

Lời khuyên: nên dùng kNN khi kích thước của bộ DL nhỏ đến vừa phải, nếu tập DL lớn thì nên dùng thuật toán khác.